<a href="https://colab.research.google.com/github/jsedoc/SPUR-Fall-2023-ChatGPT/blob/main/Trust_Game_with_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
import json
with open('/content/drive/My Drive/api_keys.json') as f:
    api_keys = json.load(f)
print(api_keys.keys())
openai.api_key = api_keys['openai_api_key']

dict_keys(['google_nlp_api_key', 'ibm_nlu_api_key', 'openai_api_key'])


In [ ]:
#https://platform.openai.com/docs/models/overview
openai_models = {
    'chatgpt': 'gpt-3.5-turbo',
    'gpt-4': 'gpt-4',
    'gpt-3.5' : 'text-divinci-003',
    'gpt-3' : 'text-divinci-001',
    'gpt-3-medium' : 'text-curie-001',
    'gpt-3-small' : 'text-babbage-001',
    'gpt-3-tiny' : 'text-ada-001'
}

In [ ]:
class Chatbot:
    """A chatbot that generates responses using one of OpenAI's GPT* language models."""

    def __init__(self, model='gpt-3.5-turbo', description='You are a helpful assistant.', name='assistant'):
        """
        Initializes the chatbot.

        Args:
        model (str): The name of the OpenAI language model to use.
        description (str): A description of the chatbot.
        name (str): The name of the chatbot.
        """
        self.history = [{"role": "system", "content": description}]
        self.model = model
        self.name = name

    def user_input(self, text):
        """
        Adds the user's input to the chatbot's history.

        Args:
        text (str): The user's input.
        """
        self.history.append({"role": "user", "content": text.strip()})

    def generate_response(self):
        """
        Generates a response using the chatbot's history and the OpenAI language model.

        Returns:
        str: The response generated by the chatbot.
        """
        try:
            if self.model in ['gpt-3.5-turbo', 'gpt-4']:
                # For documentation on the function see
                # https://platform.openai.com/docs/guides/chat/introduction
                # https://platform.openai.com/docs/api-reference/chat
                response = openai.ChatCompletion.create(
                    model=self.model,
                    messages=self.history
                )
                response_text = response.choices[0]['message']['content'].strip()
            else:
                prompt = "\n".join([f"{x['role'].replace('assistant', self.name)}: {x['content']}" for x in self.history]) + f'\n{self.name}:'
                # For documentation on the function see
                # https://platform.openai.com/docs/api-reference/completions
                response = openai.Completion.create(
                    model=self.model,
                    prompt=prompt,
                    temperature=0,
                    max_tokens=100,
                    top_p=1,
                    frequency_penalty=0.0,
                    presence_penalty=0.0,
                    stop=["\n"]
                    )
                response_text = response["choices"][0]["text"].strip()
        except openai.OpenAIError as e:
            response_text = ''
            print(f'Error generating response: {e}')
        self.history.append({"role": "assistant", "content": response_text})
        return response_text


In [ ]:
alice = Chatbot(name='Alice',
                model='gpt-4',
                description="""This task is about an exchange between yourself and another person to whom you are randomly
matched. You do not know this other person and you will not knowingly meet him/her.
You have been randomly assigned the role of the “sender”. The other person is in the role of the
“receiver”.
You and the receiver are both endowed with 10€.
You first decide how much of your 10€ endowment to transfer to the receiver. You can choose any
amount between 0€ and 10€. The amount you transfer is tripled (by the experimenter) before being
received by the receiver. The amount you keep is not tripled and simply remains in your possession.
The receiver then decides how much of the tripled transfer to return to you. The receiver can choose
any amount between 0€ and the tripled amount you sent. The amount the receiver returns is not
tripled.
Your final payment is the sum of the amount of your 10€ endowment you keep + the amount the receiver returns from your tripled transfer (in case you transfer
anything)
The receiver’s final payment is the sum of his/her 10€ endowment + the amount the receiver keeps from your tripled transfer (in case you transfer anything)
""")

bob = Chatbot(name='Bob',
              model='gpt-3.5-turbo',
              description="""This task is about an exchange between yourself and another person to whom you are randomly
matched. You do not know this other person and you will not knowingly meet him/her.
You have been randomly assigned the role of the “receiver”. The other person is in the role of the
“sender”.
You and the sender are both endowed with 10€.
The sender first decides how much of his/her 10€ endowment to transfer to you. The sender can
choose any amount between 0€ and 10€. The amount the sender transfers is tripled (by the
experimenter) before being received by you. The amount the sender keeps is not tripled and simply
remains in his/her possession.
You then decide how much of the tripled transfer to return to the sender. You can choose any
amount between 0€ and the tripled amount the sender transferred. The amount you return is not
tripled.
Your final payment is the sum of your 10€ endowment + the amount you keep from the sender’s tripled transfer (in case the sender transfers
anything)
The sender’s final payment is the sum of the amount of his/her 10€ endowment s/he keeps + the amount you return from the sender’s tripled transfer (in case the sender transfers anything)""")

In [ ]:
#print(alice.generate_response())
alice.user_input('How much of your 10€ endowment do you want to transfer to the receiver?')
alice_decision = alice.generate_response()
print('-'*50)
print(f'Alice decided: {alice_decision}')
print('-'*50)
bob.user_input(f'The sender decided {alice_decision}. Thus, you receive three times it.')
#bob.user_input(f'The sender decided {alice_decision}. Thus, you receive three times it. How much do you have now? (remember you were endowed with 10€)')
#print("Check Bob's math:", bob.generate_response())
bob.user_input('How much of this do you want to return to the sender?')
print('-'*50)
bob_decision = bob.generate_response()
print(f'Bob decided: {bob_decision}')
alice.user_input(f'The receiver decided: {bob_decision}')
print('-'*50)
bob.user_input(f'Why did you decide to give the sender that amount?')
print("Bob's rationale:", bob.generate_response())

--------------------------------------------------
Alice decided: As the sender, I will transfer 5€ of my 10€ endowment to the receiver. This means that the receiver will receive 15€ (5€ tripled) and I will keep 5€ for myself.
--------------------------------------------------
--------------------------------------------------
Bob decided: Thank you for the offer, as the receiver, I have decided to return 4€ to the sender.
--------------------------------------------------
Bob's rationale: I decided to return 4€ to the sender keeping 11€ for myself because I believe it is a fair split. By returning 4€, the sender will end up with a total of (5€-4€=1€) which is more than they originally had, while I keep more than half of the tripled amount. Furthermore, it encourages the sender to continue sending to someone else, and that's important if the ultimate goal is to continue the flow of money in the economy.


Adding some unit tests to check the code. Thanks to ChatGPT for partially making this.

In [ ]:
import unittest
from unittest.mock import patch, Mock

class TestChatbot(unittest.TestCase):
    def test_init(self):
        chatbot = Chatbot(model='test_model', description='Test description', name='test_bot')
        self.assertEqual(chatbot.model, 'test_model')
        self.assertEqual(chatbot.name, 'test_bot')
        self.assertEqual(chatbot.history, [{'role': 'system', 'content': 'Test description'}])

    def test_user_input(self):
        chatbot = Chatbot()
        chatbot.user_input('Hello')
        self.assertEqual(chatbot.history, [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Hello'}])

    def test_generate_response(self):
        chatbot = Chatbot(model='test_model', description='Test description', name='test_bot')
        chatbot.user_input('Hello')
        with patch('openai.Completion.create') as mock_create:
            mock_create.return_value = {'choices': [{'text': 'Hi there!'}]}
            response = chatbot.generate_response()
            mock_create.assert_called_once_with(
                model='test_model',
                prompt='system: Test description\nuser: Hello\ntest_bot:',
                temperature=0,
                max_tokens=100,
                top_p=1,
                frequency_penalty=0.0,
                presence_penalty=0.0,
                stop=["\n"]
            )
        self.assertEqual(response, 'Hi there!')
        self.assertEqual(chatbot.history, [{'role': 'system', 'content': 'Test description'}, {'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': 'Hi there!'}])


In [ ]:
loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestChatbot)

runner = unittest.TextTestRunner()
runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>